# GRAS (GSA and Simulated Annealing) for Feature Selection
## GRAS (GSA and Simulated Annealing) For Feature Selection on Churn Prediction
___

In [1]:
import sys
import os

# Get the directory of the current script or notebook
script_dir = os.path.dirname(os.path.abspath("__file__"))

# Construct the full path to the 'src' directory
src_dir = os.path.join(script_dir, '..', 'src')

# Append the 'src' directory to sys.path
sys.path.append(src_dir)

# Construct the full path to the 'data' directory
data_dir = os.path.join(script_dir, '..', 'data')

# Now, you can import the modules and access the data
from main import simulated_annealing
from GSA_implementation import GSA
#from benchmarks import F1 
from gsa_sa_iterative import gsa_sa_iterative
from generate_neighbor import generate_neighbor
# from utils import fitness_function, train_model
from utils3 import train_model


# Load your data (example)
import pandas as pd

#dataset1
#df = pd.read_csv(os.path.join(data_dir, 'churn2.csv'))

#dataset2
#df = pd.read_csv(os.path.join(data_dir, 'ChurnNormalize.csv'))

#dataset3
df = pd.read_csv(os.path.join(data_dir, 'cell2cell.csv'))

#dataset4
#df = pd.read_csv(os.path.join(data_dir, 'telecom_churn.csv'))

#dataset5
#df = pd.read_csv(os.path.join(data_dir, 'cleaned_dataset5.csv'))

In [3]:
import pandas as pd
import time  
import pandas as pd
import numpy as np
import random
from main import simulated_annealing
from datetime import datetime as dt
from utils import train_model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from GSA_implementation import GSA
from benchmarks import F1
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


#classifiers_name = "KNN"

#classifiers_name = {
#      "Random Forest": RandomForestClassifier(),
#      "SVM": SVC(),
#      "Decision Tree": DecisionTreeClassifier(),
#      "Logistic Regression": LogisticRegression(),
#      "Naive Bayes": GaussianNB(),
#      "KNN": KNeighborsClassifier(),
#      "Gradient Boosting": GradientBoostingClassifier(),
# }

In [4]:
#dataset1
# Assuming the target variable is named 'target' and all others are features
#X_train = df.drop(columns=['churn'])
#y_train = df['churn']

#dataset2
#X_train = df.drop(columns=['Churn'])
#y_train = df['Churn']

#dataset3
X_train = df.drop(columns=['Churn'])
y_train = df['Churn']

#dataset5
#X = df.drop(columns=['churn'])
#y = df['churn']

#print(X_train.head())
#print(y_train.head())

In [5]:
df.head()

,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
0,1,0.024537,0.029759,0.080292,0.001568,0.0,0.0,0.410058,0.303158,0.003157,...,0.0,0.444444,0,0.00,8,1,0,2,4,0
1,1,0.018836,0.001359,0.068127,0.000000,0.0,0.0,0.426933,0.308448,0.001353,...,0.0,0.555556,0,0.00,8,0,3,2,4,2
2,0,0.035924,0.001087,0.119221,0.000000,0.0,0.0,0.427153,0.308448,0.000000,...,0.0,0.666667,0,0.00,15,0,2,3,1,2
3,0,0.071937,0.178285,0.209246,0.007780,0.0,0.0,0.444690,0.310704,0.234551,...,0.0,0.666667,0,0.00,0,0,3,0,3,0
4,1,0.018958,0.000000,0.068127,0.000000,0.0,0.0,0.427374,0.308393,0.000000,...,0.0,1.000000,0,0.04,0,0,0,0,4,2


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['churn']),  # Features
    df['churn'],                # Target
    test_size=0.30,             # 30% for testing
    random_state=42             # For reproducibility
)


In [6]:
#mencoba algoritma baru2 (10-11-2024)
import pandas as pd
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from utils import get_classifier
from sklearn.model_selection import train_test_split


#Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 20  # Number of particles in GSA
gsa_iters = 15  # Number of GSA iterations per SA step
sa_iters =  10 #50  # Total number of SA steps
max_iters = 50  # Max iterations within each SA refinement
alpha = 0.93  # Cooling schedule parameter for SA
convergence_threshold = 0.001  # Threshold for SA early stopping
num_runs = 1 # Number of independent runs per classifier for robustness


# Define the classifiers dictionary

classifiers = {
    "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
#     "SVM": SVC(probability=True),
#     "Random Forest": RandomForestClassifier(),
#     "Logistic Regression": LogisticRegression(),
#     "Naive Bayes": GaussianNB(),
#     "Gradient Boosting": GradientBoostingClassifier()
}

# Initialize DataFrames to store results
all_results = pd.DataFrame()
avg_and_best_results = pd.DataFrame()

# Loop through classifiers
for name, clf in classifiers.items():
    classifier_results = []

    for run in range(1, num_runs + 1):
        print(f"Running Adaptive GSA-SA with {name} (Run {run})...")

        # Run Adaptive GSA-SA for feature selection
        best_solution, results = gsa_sa_iterative(
            X_train, y_train, max_iters=max_iters,
            gsa_iters=gsa_iters, sa_iters=sa_iters,
            alpha=alpha, convergence_threshold=convergence_threshold,
            clf_name=name
        )

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)

        # Add evaluation metrics for each subset of features
        for idx, row in results_df.iterrows():
            subset_features = row["Feature Set"]
            X_train_subset = X_train.iloc[:, subset_features]
            y_pred = clf.fit(X_train_subset, y_train).predict(X_train_subset)

            results_df.loc[idx, "Accuracy"] = accuracy_score(y_train, y_pred)
            results_df.loc[idx, "Precision"] = precision_score(y_train, y_pred)
            results_df.loc[idx, "Recall"] = recall_score(y_train, y_pred)
            results_df.loc[idx, "F1 Score"] = f1_score(y_train, y_pred)
            results_df.loc[idx, "AUC Score"] = roc_auc_score(y_train, clf.predict_proba(X_train_subset)[:, 1])

        # Add columns for classifier and run
        results_df["Classifier"] = name
        results_df["Run"] = run

        # Append results to main DataFrame
        all_results = pd.concat([all_results, results_df], ignore_index=True)

        # Track best metrics
        best_metric = results_df.loc[results_df["Fitness"].idxmin()]
        classifier_results.append(best_metric)

        # Calculate average metrics
        avg_metrics = results_df.mean(numeric_only=True)
        avg_metrics["Classifier"] = name
        avg_metrics["Run"] = run
        avg_metrics["Type"] = "Average"

        # Store best model metrics
        best_metrics = pd.Series(best_metric)
        best_metrics["Classifier"] = name
        best_metrics["Run"] = run
        best_metrics["Type"] = "Best Model"

        # Append to avg_and_best_results
        avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([avg_metrics]), pd.DataFrame([best_metrics])], ignore_index=True)

    # Overall averages and best models
    overall_avg_metrics = avg_and_best_results[(avg_and_best_results["Classifier"] == name) & (avg_and_best_results["Type"] == "Average")].mean(numeric_only=True)
    overall_avg_metrics["Classifier"] = name
    overall_avg_metrics["Run"] = "Overall Average"
    overall_avg_metrics["Type"] = "Average"

    overall_best_metrics = avg_and_best_results[(avg_and_best_results["Classifier"] == name) & (avg_and_best_results["Type"] == "Best Model")].mean(numeric_only=True)
    overall_best_metrics["Classifier"] = name
    overall_best_metrics["Run"] = "Overall Best"
    overall_best_metrics["Type"] = "Best Model"

    avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([overall_avg_metrics]), pd.DataFrame([overall_best_metrics])], ignore_index=True)

# Define classifier names
classifier_names = "_".join(classifiers.keys())  # Combine classifier names into one string    
    
# Save results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
all_results.to_csv(f"adaptive_gsa_sa_results_{classifier_names}_{num_runs}runs_{timestamp}.csv", index=False)
avg_and_best_results.to_csv(f"adaptive_gsa_sa_average_best_metrics_{classifier_names}_{num_runs}_{timestamp}.csv", index=False)

print("Results saved successfully.")

Running Adaptive GSA-SA with KNN (Run 1)...
Starting SA Iteration 1
GSA is optimizing  "<lambda>"
['At iteration 1 the best fitness is 0.3496014285714285']
['At iteration 2 the best fitness is 0.3496014285714285']
['At iteration 3 the best fitness is 0.35049428571428565']
['At iteration 4 the best fitness is 0.35112714285714286']
['At iteration 5 the best fitness is 0.35112714285714286']
['At iteration 6 the best fitness is 0.35112714285714286']
['At iteration 7 the best fitness is 0.35112714285714286']
['At iteration 8 the best fitness is 0.35112714285714286']
['At iteration 9 the best fitness is 0.35112714285714286']
['At iteration 10 the best fitness is 0.35112714285714286']
['At iteration 11 the best fitness is 0.35112714285714286']
['At iteration 12 the best fitness is 0.35112714285714286']
['At iteration 13 the best fitness is 0.35112714285714286']
['At iteration 14 the best fitness is 0.35112714285714286']
['At iteration 15 the best fitness is 0.35112714285714286']
Accepted new 

['At iteration 15 the best fitness is 0.3520357142857143']
Accepted new solution with fitness 0.34952
Accepted new solution with fitness 0.35015285714285715
Accepted new solution with fitness 0.35051
Accepted new solution with fitness 0.3510457142857143
Accepted new solution with fitness 0.35248999999999997
Accepted new solution with fitness 0.35248999999999997
Accepted new solution with fitness 0.3508671428571428
Accepted new solution with fitness 0.34952
Accepted new solution with fitness 0.3500557142857143
Accepted new solution with fitness 0.3496985714285714
Accepted new solution with fitness 0.3506885714285714
Accepted new solution with fitness 0.34870857142857137
Accepted new solution with fitness 0.34870857142857137
Accepted new solution with fitness 0.34753999999999996
Accepted new solution with fitness 0.34870857142857137
Accepted new solution with fitness 0.34753999999999996
Accepted new solution with fitness 0.3488871428571428
Accepted new solution with fitness 0.34852999999

Accepted new solution with fitness 0.34906571428571426
Accepted new solution with fitness 0.3484328571428571
Accepted new solution with fitness 0.3484328571428571
Accepted new solution with fitness 0.3482542857142857
Accepted new solution with fitness 0.3474428571428571
Accepted new solution with fitness 0.3476214285714285
Accepted new solution with fitness 0.34807571428571427
Accepted new solution with fitness 0.3482542857142857
Accepted new solution with fitness 0.3492442857142857
Accepted new solution with fitness 0.3474428571428571
Accepted new solution with fitness 0.34807571428571427
Accepted new solution with fitness 0.34807571428571427
Accepted new solution with fitness 0.3479785714285714
Accepted new solution with fitness 0.3451057142857143
Accepted new solution with fitness 0.3451057142857143
Accepted new solution with fitness 0.3441157142857143
Accepted new solution with fitness 0.3441157142857143
Accepted new solution with fitness 0.3449271428571428
Accepted new solution wi

Accepted new solution with fitness 0.3415028571428571
Accepted new solution with fitness 0.34185999999999994
Accepted new solution with fitness 0.3415028571428571
Accepted new solution with fitness 0.33970142857142854
Accepted new solution with fitness 0.3416814285714285
Accepted new solution with fitness 0.3393442857142857
Accepted new solution with fitness 0.3393442857142857
Accepted new solution with fitness 0.3399771428571428
Accepted new solution with fitness 0.3399771428571428
Accepted new solution with fitness 0.3407885714285714
Accepted new solution with fitness 0.3407885714285714
Starting SA Iteration 8
GSA is optimizing  "<lambda>"
['At iteration 1 the best fitness is 0.34878999999999993']
['At iteration 2 the best fitness is 0.35051']
['At iteration 3 the best fitness is 0.35060714285714284']
['At iteration 4 the best fitness is 0.35060714285714284']
['At iteration 5 the best fitness is 0.35060714285714284']
['At iteration 6 the best fitness is 0.35060714285714284']
['At ite

['At iteration 8 the best fitness is 0.34870857142857137']
['At iteration 9 the best fitness is 0.34870857142857137']
['At iteration 10 the best fitness is 0.34870857142857137']
['At iteration 11 the best fitness is 0.34870857142857137']
['At iteration 12 the best fitness is 0.34870857142857137']
['At iteration 13 the best fitness is 0.34870857142857137']
['At iteration 14 the best fitness is 0.34870857142857137']
['At iteration 15 the best fitness is 0.34870857142857137']
Accepted new solution with fitness 0.34538142857142856
Accepted new solution with fitness 0.3467285714285714
Accepted new solution with fitness 0.3460957142857143
Accepted new solution with fitness 0.34637142857142855
Accepted new solution with fitness 0.34142142857142854
Accepted new solution with fitness 0.3412428571428571
Accepted new solution with fitness 0.3428657142857143
Accepted new solution with fitness 0.34106428571428565
Accepted new solution with fitness 0.34268714285714286
Accepted new solution with fitn

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

# Define the classifiers
classifiers = {
    "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
    "SVM": SVC(probability=True),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Initialize an empty DataFrame to store average and best metrics for each run
avg_and_best_results = pd.DataFrame()

# Number of runs
num_runs = 10

# Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 30
iters = 100

# Generate a timestamp for dynamic file names
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Get classifier names for the file name
classifier_names = "_".join(classifiers.keys())

# Loop through each classifier
for name, clf in classifiers.items():
    classifier_results = []
    
    for run in range(1, num_runs + 1):
        #print(f"Running FS-SA with {name} (Run {run})...")
        print(f"Running FS-GSA_SA with {name} (Run {run})...")
        
        # Run the simulated annealing process with the current classifier
        #results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name=name, run_index=run)
        
        # Run GSA to get the best solution
        gsa_solution = GSA(objf=F1, lb=lb, ub=ub, dim=dim, PopSize=PopSize, iters=iters, df=df)
        initial_solution = gsa_solution.gBest

        # Run Simulated Annealing with the initial solution from GSA
        results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name = name, run_index = run, initial_solution=initial_solution)
        
        # Add columns for the classifier name and run number
        results['Classifier'] = name
        results['Run'] = run
        
        # Append the results to the combined DataFrame
        all_results = pd.concat([all_results, results], ignore_index=True)
        
        # Store the best metric from this run
        classifier_results.append(best_metric)
        
        # Calculate the average metrics for this run
        avg_metrics = results.mean()
        avg_metrics['Classifier'] = name
        avg_metrics['Run'] = run
        avg_metrics['Type'] = 'Average'

        # Store the best model's metrics
        best_metrics = pd.Series(best_metric)
        best_metrics['Classifier'] = name
        best_metrics['Run'] = run
        best_metrics['Type'] = 'Best Model'

        # Append to the DataFrame
        avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([avg_metrics]), pd.DataFrame([best_metrics])], ignore_index=True)

    # Calculate overall averages across all runs for this classifier
    overall_avg_metrics = avg_and_best_results[(avg_and_best_results['Classifier'] == name) & (avg_and_best_results['Type'] == 'Average')].mean(numeric_only=True)
    overall_avg_metrics['Classifier'] = name
    overall_avg_metrics['Run'] = 'Overall Average'
    overall_avg_metrics['Type'] = 'Average'

    overall_best_metrics = avg_and_best_results[(avg_and_best_results['Classifier'] == name) & (avg_and_best_results['Type'] == 'Best Model')].mean(numeric_only=True)
    overall_best_metrics['Classifier'] = name
    overall_best_metrics['Run'] = 'Overall Best'
    overall_best_metrics['Type'] = 'Best Model'

    # Append overall averages to the DataFrame
    avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([overall_avg_metrics]), pd.DataFrame([overall_best_metrics])], ignore_index=True)

# Dynamic file names
output_csv = f'fs_sa_results_{classifier_names}_{num_runs}runs_{timestamp}.csv'
average_output_csv = f'fs_sa_average_best_metrics_{classifier_names}_{num_runs}runs_{timestamp}.csv'

# Save the combined results to a CSV file
all_results.to_csv(output_csv, index=False)

# Save the average and best metrics to another CSV file
avg_and_best_results.to_csv(average_output_csv, index=False)

print(f"Detailed results saved to {output_csv}")
print(f"Average and best metrics saved to {average_output_csv}")


In [ ]:
results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name ="KNN")

In [ ]:
# Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 30
iters = 100

# Run GSA to get the best solution
gsa_solution = GSA(objf=F1, lb=lb, ub=ub, dim=dim, PopSize=PopSize, iters=iters, df=df)
initial_solution = gsa_solution.gBest

# Run Simulated Annealing with the initial solution from GSA
results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, initial_solution=initial_solution)


In [ ]:
results

In [ ]:
best_metric

In [ ]:
best_subset_cols

In [ ]:
# Multiple Runs for Robustness Testing
num_runs = 10  # Number of runs
run_times = []
all_metrics = []
selected_features_count = []


classifier_name = "KNN"
fitness_function = "accuracy"


for run in range(num_runs):
    print(f"Starting Run {run + 1}")
    
    # Start the timer for the run
    start_time = time.time()
    
    # Run the simulated annealing process
    results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name, fitness_function, run_index=run+1)
    
    # End the timer for the run
    end_time = time.time()
    
    # Calculate the run time and store it
    run_time = end_time - start_time
    run_times.append(run_time)
    
    print(f"Run {run + 1} completed in {run_time:.2f} seconds")
    
    
    #store the best metric from this run
    all_metrics.append(best_metric)
    # Optionally store the results
    # all_results.append((results, best_metric, best_subset_cols))
    
    # Store the number of features selected in this run
    selected_features_count.append(len(best_subset_cols))
    

# After all runs are complete
mean_runtime = np.mean(run_times)
std_runtime = np.std(run_times)

print(f"Average run time over {num_runs} runs: {mean_runtime:.2f} seconds")
print(f"Standard deviation of run time: {std_runtime:.2f} seconds")

# Calculate average and standard deviation of each metric
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
std_metrics = {key: np.std([metric[key] for metric in all_metrics]) for key in all_metrics[0]}

# Display average and standard deviation of metrics
print("\nAverage Metrics over all runs:")
for metric, value in avg_metrics.items():
    print(f"{metric}: {value:.3f}")

print("\nStandard Deviation of Metrics over all runs:")
for metric, value in std_metrics.items():
    print(f"{metric}: {value:.3f}")

# Display the best metrics for each run
print("\nBest Metrics from each run:")
for run_index, metrics in enumerate(all_metrics):
    print(f"Run {run_index + 1}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.3f}")

# Calculate and display the average number of features selected
average_features_selected = np.mean(selected_features_count)
print(f"\nAverage number of features selected over {num_runs} runs: {average_features_selected:.2f}")


In [ ]:
results